In [1]:
import json
import web3

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract

In [4]:
# Solidity source code
contract_source_code = '''
pragma solidity >0.4.0;

contract simplestorage {
  uint public storedData;

  constructor(uint initVal) public {
    storedData = initVal;
  }

  function set(uint x) public {
    storedData = x;
  }

  function get() public view returns (uint retVal) {
    return storedData;
  }
}
'''
# Compiled source code
compiled_sol = compile_source(contract_source_code) 
contract_interface = compiled_sol['<stdin>:simplestorage']

In [5]:
contract_interface['bin']

'608060405234801561001057600080fd5b506040516020806101618339810180604052810190808051906020019092919050505080600081905550506101178061004a6000396000f3006080604052600436106053576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff1680632a1afcd914605857806360fe47b11460805780636d4ce63c1460aa575b600080fd5b348015606357600080fd5b50606a60d2565b6040518082815260200191505060405180910390f35b348015608b57600080fd5b5060a86004803603810190808035906020019092919050505060d8565b005b34801560b557600080fd5b5060bc60e2565b6040518082815260200191505060405180910390f35b60005481565b8060008190555050565b600080549050905600a165627a7a72305820a70f3b71e0e350d0c7cef607d854d19ad5fb59bd743ca1120b6fa21be9467e900029'

In [54]:
NODE_ADDR = 'http://nd-938-939-954.rg-837-380.p2pify.com:8545'

In [55]:
# web3.py instance
w3 = Web3(Web3.HTTPProvider(NODE_ADDR))

In [56]:
# inject the poa compatibility middleware to the innermost layer
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

In [57]:
w3.version.node

'Geth/v1.8.12-stable-c6f0ae4e/linux-amd64/go1.9.7'

In [58]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]
'Account:', w3.eth.defaultAccount

('Account:', '0x9A1A60Ff9c31d5D82022c31d30CeD203411210d7')

In [59]:
# Instantiate and deploy contract
simplestorage = w3.eth.contract(
    abi=contract_interface['abi'],
    bytecode=contract_interface['bin']
)

In [60]:
# Submit the transaction that deploys the contract
tx_hash = simplestorage.constructor(42).transact()

In [61]:
tx_hash

HexBytes('0xbc00b72be8a0fcd0c301539c6082ce5949a22decfbde90b660b6ff7e54d5c39b')

In [62]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [63]:
'Block number:', tx_receipt.blockNumber

('Block number:', 1)

In [64]:
'Contract address:', tx_receipt.contractAddress

('Contract address:', '0x354BBcBA06825055ee952De5B0B84177582fe64e')

In [65]:
simplestorage = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [66]:
'Storage value:', simplestorage.functions.get().call()

('Storage value:', 42)

In [67]:
tx_hash = simplestorage.functions.set(128).transact()

In [68]:
tx_hash

HexBytes('0x940315be573851f99b887d57a1fcaccddb463f28ef6de54c3daf0d261fe22a96')

In [69]:
# Wait for transaction to be mined...
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [70]:
'Block number:', tx_receipt.blockNumber

('Block number:', 2)

In [ ]:
'Storage value:', simplestorage.functions.get().call()